<a href="https://colab.research.google.com/github/hiew1/SeaCodeBankWebApp/blob/master/Copy_of_Super7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
from keras.utils import to_categorical

X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)

input_shape =(32, 32,3)

# one hot encode target values
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

def init():
    model  =Sequential()
    #model.add(Conv2D(28, kernel_size=(3, 3), input_shape = input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    #loss='mean_squared_error'
    return model

In [ ]:
def train(models):
    losses = []
    histories = []
    for i in range(len(models)):
        history = models[i].fit(x=X_train,y=y_train, epochs=1, validation_data=(X_test, y_test))
        losses.append(round(history.history['loss'][-1], 4))
        histories.append(history)
    return models, losses, histories

In [ ]:
# Depending on the Application the number of generations may or may not be fixed
no_of_generations = 1

# Must be set to more than two for crossover to work
no_of_individuals = 2
mutation_rate = 0.05
individuals = []

In [ ]:
#model = init()

#model_history = model.fit(x=X_train,y=y_train, epochs=1, validation_data=(X_test, y_test), batch_size=64)

In [ ]:
import random

# Initialize Population
def randomized_init():
    model=Sequential()
    
    # Model always starts with one convolutional layer and one pooling later right after it
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Randomize number of variable layers between the fixed layers, ranging between 0 and 5
    numberofVariableLayers = random.randrange(0, 6)
    
    for i in range(numberofVariableLayers):
        # Randomize whether to add Convolutional Layer or Max Pooling Layer
        randomLayer = random.randrange(1)
        
        # Randomize strides from 1 to 3
        randomStrides = random.randrange(1, 4)
        
        if(randomLayer == 0):
            model.add(keras.layers.Conv2D(filters = 32, kernel_size = (3, 3), strides=randomStrides, padding='same', activation='relu', input_shape=(32,32,3)))
            #model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
        else:
            model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Model always ends with flatten, dense, dropout and dense
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    #loss='mean_squared_error'
    return model

for i in range(no_of_individuals):
    individuals.append(randomized_init())
    
#for model_randomized in individuals:
#    print(model_randomized.summary())



In [ ]:
#for generation in range(no_of_generations):
#    print(f"Generation: {generation}")
#    individuals, losses, histories = train(individuals)
#    print(histories)

In [ ]:
def crossover(individuals):
    new_individuals = []
    # Choose the two fittest ones
    new_individuals.append(individuals[0])
    new_individuals.append(individuals[1])

    for i in range(0, len(individuals)):
      # Pick two random individuals
      random_individualA = random.randrange(0, len(individuals))
      random_individualB = random.randrange(0, len(individuals))

      # Pick two random numbers representing two random positions of layers between the 3rd (counting from the first layer), and the 5th (counting from last layer)
      random_layerA = random.randrange(3, len(individuals[random_individualA].layers) - 4)
      random_layerB = random.randrange(3, len(individuals[random_individualB].layers) - 4)

      temp_gene = individuals[random_individualA].layers[random_layerA].get_weights()[1]
      individuals[random_individualA].layers[random_layerA].get_weights()[1] = individuals[random_individualB].layers[random_layerB].get_weights()[1]
      individuals[random_individualB].layers[random_layerB].get_weights()[1] = temp_gene

      # Pick just one random individual from the pair of parents to survive into next generation
      new_individual = random.choice([individuals[random_individualA], individuals[random_individualB]])

      new_individuals.append(new_individual)

    return new_individuals

In [ ]:
def mutate(individuals):
    new_individual = []
    # Mutate a random weight factor based on a mutation rate
    for individual in individuals:
      if (random.uniform(0, 1) < mutation_rate):
        # Pick a random number representing a random position of a layer between the 3rd (counting from the first layer), and the 5th (counting from last layer)
        random_layer_to_mutate = random.randrange(3, len(individual.layers) - 4)
        individual.layers[random_layer_to_mutate].get_weights()[1] = individual.layers[random_layer_to_mutate].get_weights()[1] * random.uniform(-0.5, 0.5)
        new_individual.append(individual)
      else:
        new_individual.append(individual)

    return new_individual

In [ ]:
def evolve(individuals, losses):
    sorted_by_losses_fitness_evaluation = sorted(range(len(losses)),key=lambda x:losses[x])
    individuals = [individuals[i] for i in sorted_by_losses_fitness_evaluation]

    #winners = individuals[:6]

    new_individuals = crossover(individuals)

    new_individuals = mutate(individuals)

    return new_individuals

In [ ]:
for generation in range(no_of_generations):
    individuals, losses, histories = train(individuals)

    individuals = evolve(individuals, losses)

1563/1563 [==============================] - 58s 37ms/step - loss: 1.5192 - accuracy: 0.4511 - val_loss: 1.2347 - val_accuracy: 0.5499


AttributeError: ignored